## 25.3 Dynamic programming

Dynamic programming is often applied to optimisation problems on sequences,
because sequences are usually easy to process item by item to build up
the problem's solution from the subproblems' solutions.

To obtain a dynamic programming algorithm, follow these steps.

1. **Define the problem recursively.**

This is the key step and usually the hardest one.

The recursive definition has the same usual form.
First define the base cases, which include the smallest inputs but possibly
other cases too. Then define the recurrence relations: how the solution for
a problem instance is obtained from solutions for subproblems
(smaller input instances). For problems on sequences,
use the head, tail and concatenation operations.

For optimisation problems, the recurrence relation must involve taking
the best (typically the maximum or minimum) of two or more subproblem solutions.
In the [LCS problem](../23_Dynamic_Programming/23_2_lcs.ipynb#23.2.1-Recursive),
if the two string heads differ, we take the longest of two subsequences,
by skipping the left or the right head.

1. **Implement the recursive definition.**

Implement the recursive definition, which is usually straightforward,
using `...[0]` for the head of a sequence and `...[1:]` for the tail.
If the problem turns out to have overlapping subproblems, then
the recursive algorithm may be exponential in the worst case, so
test your code on small problem instances.

Once you have a working implementation,
modify it to use indices instead of slicing.
This improves the run-time and helps introduce a cache later.

When using indices, a recursive definition of function f on sequence *items* like

- if *items* is empty: f(*items*) = ...
- otherwise: f(*items*) = ... head(*items*) ... tail(*items*) ...

becomes a definition for a function f with an an additional integer parameter *index*:

- if *index* = │*items*│: f(*items*, *index*) = ...
- otherwise: f(*items*, *index*) = ... *items*[*index*] ... *index* + 1 ...

The following code template uses an auxiliary inner recursive function that
has only parameter *index* and accesses *items* from the main function.
```python
def f(items: list) -> ...:
    def auxiliary(index: int) -> ...:
        if index == len(items):
            return ...
        else:
            return ... # based on items[index] and index+1

    return auxiliary(0) # start at index 0
```
3. **Are there overlapping subproblems?**

The next step is to show the existence of repeatedly solved subproblems,
to justify the need for dynamic programming.
You have to think whether it's possible to arrive at the same subproblem
by making different choices whilst processing the input.
It may help to draw the tree of recursive calls for a small problem instance
to see if repeated nodes appear.

For example, in the [LCS problem](../23_Dynamic_Programming/23_2_lcs.ipynb#23.2.2-Top-down),
whether we first skip the left head and then the right head or vice versa,
we obtain the same subproblem. In the knapsack problem,
[if two or more items have the same weight](../23_Dynamic_Programming/23_3_knapsack.ipynb#23.3.2-Common-subproblems),
choosing any of them and skipping the others will lead to the same subproblem:
the same remaining capacity and the same remaining items to choose from.

4. **Define a cache data structure.**

The subproblems are the possible value combinations for the inputs.
The cache is a map of subproblems to their solutions.

If the inputs are one, two, three or more natural numbers, the cache
can be implemented as a one-, two-, three- or higher-dimensional array,
indexed by the input numbers. For example, the cache for the
[LCS problem](../23_Dynamic_Programming/23_2_lcs.ipynb#23.2.2-Top-down-with-matrix) is
a two-dimensional array (a matrix) indexed by the indices of
the left and right input strings.

If the input values are hashable, like a tuple or a string, the cache can be
implemented with a hash table (a Python dictionary).
If the input values are comparable but not hashable (like Python lists),
consider a binary search tree for logarithmic lookup.
You may also consider converting to a hashable data type, like tuples.
If there's no efficient representation for the cache you need,
consider going back to step&nbsp;1 to find an alternative problem with inputs that
suit dynamic programming better.

5. **Implement top-down dynamic programming.**

The next step is to modify the code of step&nbsp;2 with the cache defined in step&nbsp;4.
The required modifications are usually rote. The original function
```python
def f(instance: ...) -> ...:
    if instance is base case:
        return ...
    else:
        return ...
```
becomes
```python
def f(instance: ...) -> ...:

    def auxiliary(instance: ...) -> ...:
        if instance not in cache:
            if instance is base case:
                cache[instance] = ...
            else:
                cache[instance] = ...
        return cache[instance]

    cache = ... # initialise cache
    return auxiliary(instance)
```
assuming the cache is a dictionary or array.
If the cache is a dictionary, start it empty; if it's an array, initialise it
with an impossible value that can't be the solution to any subproblem.

6. **Implement bottom-up dynamic programming.**

To develop the bottom-up approach, look at the recurrence relations or the code
of the top-down approach to understand in which order to fill the cache.
It may help to draw a DAG of the subproblems and their dependencies
for a particular problem instance.

The DAG may be easiest to draw by thinking backwards, basically following
the recursive call tree and merging common subproblems into one node.
However, you may also draw the DAG starting with the base cases and proceeding
in a breadth-first way: Which subproblems depend only on the base cases?
Which subproblems then depend on the ones just added?
In other words, think which subproblems 'feed' into which other ones.

The subproblems can then be solved in any topological order of the DAG.
If the cache is a multi-dimensional array,
it may be as simple to fill as using nested loops, one per dimension.
Each loop goes through the corresponding indices in ascending (first to last)
or descending (last to first) order. The code becomes:
```python
def f(instance: ...) -> ...:
    cache = ... # initialise cache as in top-down approach

    for row in range(...):
        for column in range(...):
            if the instance (row, column) is a base case:
                cache[row][column] = ...
            else:
                cache[row][column] = ...

    return cache[...][...]  # usually one of the corner cells
```
After filling the cache, the algorithm returns the entry corresponding to
the input instance.

For example, in the [LCS problem](../23_Dynamic_Programming/23_2_lcs.ipynb#23.2.5-Bottom-up)
each matrix cell depends on cells below or to the right in the same row, so
both the row and column loops must iterate backwards, in descending index order,
and the solution for the input instance is in the top left-hand cell.

However, for the knapsack problem, each cell only depends on
[cells below and to the left](../23_Dynamic_Programming/23_3_knapsack.ipynb#23.3.3-Top-down-and-bottom-up),
so we iterate through the rows in descending order and
through the columns in ascending order.
The final solution is in the top right-hand cell.

7. **Analyse the complexity and measure the performance.**

The worst-case complexity of dynamic programming (either approach)
is the size of the cache multiplied by the worst-case complexity of
computing each entry. The size of the cache is the number of subproblems,
which is at most the product of all possible values for each input.

The top-down approach fills only the part of the cache that is needed for
the input problem instance, while bottom-up fills the whole cache.
On the other hand, the top-down algorithm makes recursive calls,
while bottom-up is purely iterative.
Even though the worst-case complexity is the same,
the bottom-up approach may have lower run-times if
the top-down approach fills most of the cache.
You will have to measure the run-times of both approaches for typical inputs
to assess which approach is best in practice.

⟵ [Previous section](25_2_backtracking.ipynb) | [Up](25-introduction.ipynb) | [Next section](../26_Complexity_classes/26-introduction.ipynb) ⟶